In [3]:
import sys
import os
import pandas as pd
current_file_path = os.path.abspath(os.getcwd())
parent_dir = os.path.abspath(os.path.join(current_file_path, '..'))
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

from argparse import Namespace
from constants.paths import FOLDER_PATH
from examples.benchmark import local_get_args
from load_inputs.bike_in import load_data, START, END
from plotting.TS_analysis import plot_TS



coverage_period = pd.date_range(start=START, end=END, freq='15min')[:-1]  # Exclude the last date to match the data
invalid_dates = []
minmaxnorm = True
standardize = False  # Set to True if you want to standardize the data
config = {'model_name': 'STGCN',
         'freq' : '15min',
        'dataset_names':['bike_in'],
        'dataset_for_coverage' : ['bike_in'],
        'target_data': 'bike_in',
        'contextual_kwargs':{'bike_in':{'agg_iris_target_n':50,
                                        'threshold_volume_min': 1,
                                },
                                        },
        'step_ahead':1,
        'horizon_step':1,
        }

args = local_get_args(config['model_name'],
                args_init = None,
                dataset_names=config['dataset_names'],
                dataset_for_coverage=config['dataset_for_coverage'],
                modification = config)


ds = load_data(FOLDER_PATH, coverage_period, invalid_dates, args, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)

df =  pd.DataFrame(ds.U_train[:,:,-1].detach().cpu().numpy(),columns = ds.spatial_unit, index = ds.tensor_limits_keeper.df_verif_train.iloc[:,-2] )
plot_TS(df.iloc[:2000,:],bool_show = True, height = 800)

Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_attracted_by_station15min.csv...
df pivoted:  (70094, 435)
df reindexed :  (35040, 435)
Len coverage period:  35040
df filtered:  (35040, 435)
   Loaded data: (35040, 435)
   Dimension after spatial agg: (35040, 13)


Loading BokehJS ...

figure(id='p1579', ...)

In [4]:
from load_inputs.bike_out import load_data as load_data_out

args_out = Namespace(**vars(args))  
args_out.target_data = 'bike_out'
args_out.dataset_for_coverage = ['bike_out']
args_out.dataset_names = ['bike_out']
args_out.contextual_kwargs = {'bike_out':{'agg_iris_target_n':50,
                                          'threshold_volume_min': 1,
                                         
                                        },
                            }


ds_out = load_data_out(FOLDER_PATH, coverage_period, invalid_dates, args_out, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)
df_out =  pd.DataFrame(ds_out.U_train[:,:,-1].detach().cpu().numpy(),columns = ds_out.spatial_unit, index = ds_out.tensor_limits_keeper.df_verif_train.iloc[:,-2] )


# Filtered
args_out.contextual_kwargs = {'bike_out':{'agg_iris_target_n':50,
                                          'threshold_volume_min': 1,
                                           'quantile_filter_outliers': 0.98
                                        },
                            }
ds_out_filtered = load_data_out(FOLDER_PATH, coverage_period, invalid_dates, args_out, minmaxnorm,standardize, normalize=True,
              tensor_limits_keeper = None)
df_out_filtered =  pd.DataFrame(ds_out_filtered.U_train[:,:,-1].detach().cpu().numpy(),columns = [f"filtered_{c}" for c in ds_out_filtered.spatial_unit], index = ds_out_filtered.tensor_limits_keeper.df_verif_train.iloc[:,-2] )
# ---

df_out_concat = pd.concat([df_out, df_out_filtered], axis=1)
plot_TS(df_out_concat.iloc[:2000,:],bool_show = True, height = 800)

Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 13)
Loading from /home/rrochas/../../data/rrochas/prediction_validation/agg_data/velov/velov_emitted_by_station15min.csv...
df pivoted:  (70049, 434)
df reindexed :  (35040, 434)
Len coverage period:  35040
df filtered:  (35040, 434)
   Loaded data: (35040, 434)
   Dimension after spatial agg: (35040, 13)


Loading BokehJS ...

figure(id='p1784', ...)